In [1]:
from fastai.vision import *

In [10]:
def res_block_with_depth(n_in, n_out, n_blocks=1, depth=1, dense:bool=False, norm_type:Optional[NormType]=NormType.Batch, **conv_kwargs):
    "Resnet block of `nf` features. `conv_kwargs` are passed to `conv_layer`."
    norm2 = norm_type
    if not dense and (norm_type==NormType.Batch): norm2 = NormType.BatchZero
    layer1 = [conv_layer(n_in, n_out, norm_type=norm_type, stride=(2, 2), **conv_kwargs)]
    layers = [res_block(n_out, depth=depth) for _ in range(n_blocks)]
    if len(layers) == 0: return layer1
    return SequentialEx(*(layer1+layers))

def res_block(nf, depth=1, dense:bool=False, norm_type:Optional[NormType]=NormType.Batch, **conv_kwargs):
    "Resnet block of `nf` features. `conv_kwargs` are passed to `conv_layer`."
    norm2 = norm_type
    if not dense and (norm_type==NormType.Batch): norm2 = NormType.BatchZero
    layers = []
    for i in range(depth):
        layers.append(conv_layer(nf, nf, norm_type=norm2, **conv_kwargs))
    layers.append(MergeLayer(dense))
    return SequentialEx(*layers)

In [2]:
from models.resnets import *

In [3]:
get_resnet(n_blocks=5, n_basic_blocks=0)

SequentialEx(
  (layers): ModuleList(
    (0): Sequential(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): ReLU(inplace)
      (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (2): SequentialEx(
      (layers): ModuleList(
        (0): Sequential(
          (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (1): ReLU(inplace)
          (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (1): Sequential(
          (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (1): ReLU(inplace)
          (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (2): MergeLayer()
      )
    )
    (3): SequentialEx(
      (layers): ModuleList(
 

In [11]:
res_block_with_depth(1, 2, n_blocks=2, depth=2)

SequentialEx(
  (layers): ModuleList(
    (0): Sequential(
      (0): Conv2d(1, 2, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): ReLU(inplace)
      (2): BatchNorm2d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): SequentialEx(
      (layers): ModuleList(
        (0): Sequential(
          (0): Conv2d(2, 2, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (1): ReLU(inplace)
          (2): BatchNorm2d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (1): Sequential(
          (0): Conv2d(2, 2, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (1): ReLU(inplace)
          (2): BatchNorm2d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (2): MergeLayer()
      )
    )
    (2): SequentialEx(
      (layers): ModuleList(
        (0): Sequential(
          (0): Conv2d(2, 2, kernel_size=(3, 3), stride=(1, 1), paddi

In [44]:
res_block_with_depth(1, 1, 2)

SequentialEx(
  (layers): ModuleList(
    (0): Sequential(
      (0): Conv2d(1, 1, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): ReLU(inplace)
      (2): BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): SequentialEx(
      (layers): ModuleList(
        (0): Sequential(
          (0): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (1): ReLU(inplace)
          (2): BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (1): MergeLayer()
      )
    )
  )
)

In [45]:
new_net = nn.Sequential(
    conv_layer(3, 64, ks = 7, stride = 2, padding = 3),
    nn.MaxPool2d(3, 2, padding = 1),
    res_block(64, depth=2),
    res_block_with_depth(64, 128, 2),
    res_block_with_depth(128, 265, 2),
    res_block_with_depth(256, 512, 2),
    create_head(1024, 10, ps=0)
)

In [62]:
def get_resnet(input_channels=3, out_classes=10, depth=2, n_blocks=3):
    net = [conv_layer(3, 64, ks = 7, stride = 2, padding = 3),
            nn.MaxPool2d(3, 2, padding = 1),
            res_block(64, depth=2)]
    in_c = net[0][0].out_channels
    for i in range(n_blocks):
        net.append(res_block_with_depth(in_c, 2*in_c, depth=depth))
        in_c *= 2
    net.append(create_head(in_c*2, 10, ps=0))
    return SequentialEx(*net)

    


In [63]:
get_resnet()

SequentialEx(
  (layers): ModuleList(
    (0): Sequential(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): ReLU(inplace)
      (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (2): SequentialEx(
      (layers): ModuleList(
        (0): Sequential(
          (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (1): ReLU(inplace)
          (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (1): Sequential(
          (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (1): ReLU(inplace)
          (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (2): MergeLayer()
      )
    )
    (3): SequentialEx(
      (layers): ModuleList(
 

In [60]:
la.out_channels

64

In [54]:
nn.Sequential(*net).cuda().forward(torch.randn(2, 3, 224, 224).cuda()).shape

torch.Size([2, 10])

In [6]:
    res_block(265)

SequentialEx(
  (layers): ModuleList(
    (0): Sequential(
      (0): Conv2d(265, 265, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): ReLU(inplace)
      (2): BatchNorm2d(265, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): Sequential(
      (0): Conv2d(265, 265, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): ReLU(inplace)
      (2): BatchNorm2d(265, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (2): MergeLayer()
  )
)

In [12]:
res_block(14, bottle=True)

SequentialEx(
  (layers): ModuleList(
    (0): Sequential(
      (0): Conv2d(14, 7, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): ReLU(inplace)
      (2): BatchNorm2d(7, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): Sequential(
      (0): Conv2d(7, 14, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): ReLU(inplace)
      (2): BatchNorm2d(14, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (2): MergeLayer()
  )
)

In [3]:
def res_block(n_in, n_out, dense:bool=False, norm_type:Optional[NormType]=NormType.Batch, **conv_kwargs):
    "Resnet block of `nf` features. `conv_kwargs` are passed to `conv_layer`."
    norm2 = norm_type
    if not dense and (norm_type==NormType.Batch): norm2 = NormType.BatchZero
    return SequentialEx(conv_layer(n_in, n_out, norm_type=norm_type, **conv_kwargs),
                      conv_layer(n_out, nf, norm_type=norm2, **conv_kwargs),
                      MergeLayer(dense))

res_block(n_in=64, n_out=128)

Sequential(
  (0): AdaptiveConcatPool2d(
    (ap): AdaptiveAvgPool2d(output_size=1)
    (mp): AdaptiveMaxPool2d(output_size=1)
  )
  (1): Flatten()
  (2): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (3): Dropout(p=0.25)
  (4): Linear(in_features=1024, out_features=512, bias=True)
  (5): ReLU(inplace)
  (6): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (7): Dropout(p=0.5)
  (8): Linear(in_features=512, out_features=10, bias=True)
)

In [14]:
import torchvision
import torch

In [16]:
from torchvision import datasets

In [23]:
ls = []

In [25]:
len(ls)

0

In [76]:
a = torch.randn(3, 224, 224)

In [80]:
adaptive_pooling = nn.AdaptiveMaxPool2d(output_size=1)

In [81]:
adaptive_pooling(a)

tensor([[[4.3917]],

        [[3.8662]],

        [[4.1667]]])

In [85]:
a[1].max()

tensor(3.8662)

In [1]:
from models.resnets import get_resnet
from models.unet import DynamicUnet

In [2]:
import torch

In [3]:
net = get_resnet(fc=False)

In [26]:
net

SequentialEx(
  (layers): ModuleList(
    (0): Sequential(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): ReLU(inplace)
      (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (2): SequentialEx(
      (layers): ModuleList(
        (0): Sequential(
          (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (1): ReLU(inplace)
          (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (1): Sequential(
          (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (1): ReLU(inplace)
          (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (2): MergeLayer()
      )
    )
    (3): SequentialEx(
      (layers): ModuleList(
 

In [10]:
unet = DynamicUnet(net, n_classes=10).cuda()

In [8]:
from fastai.vision import *

In [11]:
unet

DynamicUnet(
  (layers): ModuleList(
    (0): SequentialEx(
      (layers): ModuleList(
        (0): Sequential(
          (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
          (1): ReLU(inplace)
          (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (1): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
        (2): SequentialEx(
          (layers): ModuleList(
            (0): Sequential(
              (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
              (1): ReLU(inplace)
              (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            )
            (1): Sequential(
              (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
              (1): ReLU(inplace)
              (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runn

In [12]:
from torchsummary import summary

In [22]:
summary(unet, input_size=(3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
              ReLU-2         [-1, 64, 112, 112]               0
       BatchNorm2d-3         [-1, 64, 112, 112]             128
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
              ReLU-6           [-1, 64, 56, 56]               0
       BatchNorm2d-7           [-1, 64, 56, 56]             128
            Conv2d-8           [-1, 64, 56, 56]          36,864
              ReLU-9           [-1, 64, 56, 56]               0
      BatchNorm2d-10           [-1, 64, 56, 56]             128
       MergeLayer-11           [-1, 64, 56, 56]               0
     SequentialEx-12           [-1, 64, 56, 56]               0
           Conv2d-13          [-1, 128, 28, 28]          73,728
             ReLU-14          [-1, 128,

In [14]:
from fastai.vision.models import DynamicUnet as DUnet

In [20]:
new = DUnet(create_body(models.resnet34), n_classes=10).cuda()

In [16]:
from fastai.vision import *

In [18]:
new

DynamicUnet(
  (layers): ModuleList(
    (0): Sequential(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace)
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (4): Sequential(
        (0): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (1): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum

In [24]:
from torchviz import make_dot

In [25]:
new.parameters()

<generator object Module.parameters at 0x7fc3d5f6b6d0>

In [28]:
y = net(torch.randn(1, 3, 224, 224).cuda())

In [32]:
make_dot(y)

ExecutableNotFound: failed to execute ['dot', '-Tsvg'], make sure the Graphviz executables are on your systems' PATH